In [127]:
import sqlite3
import os.path
from datetime import datetime
import re
import codecs
con = sqlite3.connect("seminar.db")
cur = con.cursor()

In [128]:
text_file = open("src/seminar.txt", "r");
seminar_template = text_file.read();
text_file.close();

text_file = open("src/seminars.txt", "r");
seminars_template = text_file.read();
text_file.close();

In [138]:
def makeLink(text, link):
    if (link is not None):
        return '<a href="'+link+'">'+text+'</a>';
    else:
        return text;

def displaySeminar(template, seminar):
    linkline="<li>\n\t{{DATE}} {{LOCATION}} <br />\n\t{{SPEAKER}} {{INSTITUTION}} <br />\n\t{{TITLE}}\n</li>\n";
    if (seminar[8] is None):
        title="TBD";
    else:
        title=seminar[8];
    template=template.replace("{{SEMINAR_TITLE}}", title);
    link=makeLink(seminar[4], seminar[5]);
    template=template.replace("{{SPEAKER}}", link);
    institution='';
    if (seminar[6] is not None):
        institution=makeLink(seminar[6], seminar[7]);
    template=template.replace("{{INSTITUTION}}", institution);
    abstract='';
    if (seminar[9] is not None):
        abstract=seminar[9];
    abstract=abstract.replace("\\r\\n", "<br />");
    template=template.replace("{{ABSTRACT}}",abstract);
    datetime_object = datetime.strptime(seminar[1], '%Y-%m-%d')
    template=template.replace("{{DATE}}",datetime_object.strftime("%A %d %b %Y"));
    template=template.replace("{{LOCATION}}",seminar[3]);
    filename=re.sub(r'[^A-Za-z0-9 ]+', '', title);
    filename="site/seminars/"+seminar[1]+'-'+filename.replace(" ", "_")+".html";
    out_file = codecs.open(filename, "w", "utf-8");
    out_file.write(template);
    out_file.close();
    linkline=linkline.replace("{{DATE}}",datetime_object.strftime("%d %b %Y"));
    linkline=linkline.replace("{{LOCATION}}", seminar[3]);
    linkline=linkline.replace("{{SPEAKER}}", link);
    linkline=linkline.replace("{{INSTITUTION}}", institution);
    linkline=linkline.replace("{{TITLE}}", makeLink(title, filename));
    linkline=linkline.replace("site/seminars","../seminars");
    return linkline; #return link to be used
    
    
def writeYearList(yy, year_list, seminars_data):
    filename="site/years/"+yy+".html";
    out_file = codecs.open(filename, "w", "utf-8");
    out_file.write(seminars_data.replace("{{TALKLIST}}", year_list).replace("{{HEADING}}",yy));
    out_file.close();

def seminarsFromYear(year, seminar_template, seminars_template):
    query="SELECT * FROM seminar where date>='{{yy1}}-10-01' and date <='{{yy2}}-09-30' order by date asc";
    query=query.replace("{{yy1}}",str(year)).replace("{{yy2}}",str(year+1));
    res=cur.execute(query);

    year_list='';

    for seminar in res:
        linkline=displaySeminar(seminar_template, seminar);
        year_list=year_list+linkline;

    if (year_list != ''):
        writeYearList("{{yy1}}-{{yy2}}".replace("{{yy1}}",str(year)).replace("{{yy2}}",str(year+1)), year_list, seminars_template);
        
def makeYearItem(inYearDir, year):
    query="SELECT * FROM seminar where date>='{{yy1}}-10-01' and date <='{{yy2}}-09-30' ";
    query=query.replace("{{yy1}}",str(year)).replace("{{yy2}}",str(year+1));
    res=cur.execute(query);
    if (len(res.fetchall())>0):
        listitem="\t<li>{{LINK}}</li>\n";
        if (inYearDir==True):
            listitem=listitem.replace("{{LINK}}", makeLink(str(year)+'-'+str(year+1), str(year)+'-'+str(year+1)+".html"));
        else:
            listitem=listitem.replace("{{LINK}}", makeLink(str(year)+'-'+str(year+1), "../years/"+str(year)+'-'+str(year+1)+".html"));
        return listitem;
    else:
        return '';
    
def makeYearList(inYearDir):
    yearlist='<ul>\n';
    for i in range (1993,2023):
        yearlist=yearlist+makeYearItem(inYearDir, i);
    return yearlist+'</ul>\n'

def writeIndexHTML(seminars_data):
    filename="site/index.html";
    seminars_data=seminars_data.replace("../resources/", "resources/");
    seminars_data=seminars_data.replace('<li><a href="', '<li><a href="years/');
    out_file = codecs.open(filename, "w", "utf-8");
    out_file.write(seminars_data.replace("{{TALKLIST}}", "").replace("{{HEADING}}",""));
    out_file.close();
    

In [130]:
yearListinYears=makeYearList(True);
yearListinSeminars=makeYearList(False);
seminar_template=seminar_template.replace("{{YEARLIST}}", yearListinSeminars);
seminars_template=seminars_template.replace("{{YEARLIST}}", yearListinYears);

In [131]:
for i in range (1993,2023):
    seminarsFromYear(i, seminar_template, seminars_template);


In [139]:
writeIndexHTML(seminars_template);